# Get Renewable Share Data for Past 5 Years

de - Germany
ch - Switzerland
eu - European Union
all - Europe
al - Albania
am - Armenia
at - Austria
az - Azerbaijan
ba - Bosnia-Herzegovina
be - Belgium
bg - Bulgaria
by - Belarus
cy - Cyprus
cz - Czech Republic
dk - Denmark
ee - Estonia
es - Spain
fi - Finland
fr - France
ge - Georgia
gr - Greece
hr - Croatia
hu - Hungary
ie - Ireland
it - Italy
lt - Lithuania
lu - Luxembourg
lv - Latvia
md - Moldova
me - Montenegro
mk - North Macedonia
mt - Malta
nie - North Ireland
nl - Netherlands
no - Norway
pl - Poland
pt - Portugal
ro - Romania
rs - Serbia
ru - Russia
se - Sweden
si - Slovenia
sk - Slovak Republic
tr - Turkey
ua - Ukraine
uk - United Kingdom
xk - Kosovo

In [68]:
def get_renewable_share_data(country_code, rs, l):
    import requests
    import json
    import pandas as pd

    url = "https://api.energy-charts.info/public_power?"
    country = country_code
    start= "2019-01-01"
    stop= "2025-01-01"
    req = requests.get(url + "country=" + country + "&start=" + start + "&end=" + stop)
    data = req.json()
    ren_share =pd.json_normalize(data["production_types"][rs]).explode("data").reset_index().drop(["index", "name"], axis = 1)
    ren_share = ren_share.rename(columns={'data': 'ren_share'})
    load =pd.json_normalize(data["production_types"][l]).explode("data").reset_index().drop(["index", "name"], axis = 1)
    load = load.rename(columns={'data': 'load'})
    time = pd.DataFrame(data["unix_seconds"], columns=["unix_seconds"])
    time["datetime"] = pd.to_datetime(time["unix_seconds"], unit="s")
    data_df = pd.concat([time, ren_share, load], axis = 1)
    data_df["year"] = data_df["datetime"].dt.year
    grouped_df = data_df.groupby("year")[["load", "ren_share"]].mean().reset_index()
    return grouped_df

In [82]:
import pandas as pd
countries = ["de", "fr", "dk", "nl", "pl", "ie"]
ren = [20, 15, 11, 12, 13, 12]
total = [18, 13, 9, 10, 11, 10]
for i, j, country in zip(ren, total, countries):
    df = get_renewable_share_data(country, i, j)
    df.to_csv("renewable_shares_"+country, index=False)